In [1]:
import requests
from bs4 import BeautifulSoup
import pymongo
import time
import json
%matplotlib inline
import re

In [2]:
import json
f = open("movie.json", "r")
movie_data = json.load(f)
f.close()
movie_data = movie_data
print(movie_data[0].keys())
line = ""
for i in movie_data[0:100]:
    line += "{}\t".format(i['Mname'])
print(line)

[u'realease_kor', u'Webpage', u'Mname', u'Domestic', u'budget', u'gross_kor', u'release_usa', u'year', u'runtime', u'Oversea', u'rating_usa']
Toy Story 3	Alice in Wonderland (2010)	Harry Potter and the Deathly Hallows Part 1	Inception	Shrek Forever After	The Twilight Saga: Eclipse	Iron Man 2	Tangled	Despicable Me	How to Train Your Dragon	Clash of the Titans (2010)	The Chronicles of Narnia: The Voyage of the Dawn Treader	The King's Speech	Tron Legacy	The Karate Kid	Prince of Persia: The Sands of Time	Black Swan	Megamind	Robin Hood	The Last Airbender	Little Fockers	Resident Evil: Afterlife	Shutter Island	Salt	Sex and the City 2	The Tourist	The Expendables	Knight & Day	True Grit	Gulliver's Travels	Percy Jackson & The Olympians: The Lightning Thief	The Social Network	Valentine's Day	The Sorcerer's Apprentice	Due Date	Eat Pray Love	Red	Paranormal Activity 2	The A-Team	Jackass 3-D	Unstoppable	Step Up 3-D	The Book of Eli	The Town	Legend of the Guardians: The Owls of Ga'Hoole	The Wolfman	The B

In [3]:
class Database(object):
    def __init__(self):
        self.connection = pymongo.MongoClient("localhost", 27017)
        self.db = self.connection["DSS4_SmallP"]
        self.collection = self.db["Movie_mojo"]        
def Load_Data_mojo_yearly_dom(page,year):
    Mojo_url = u'http://www.boxofficemojo.com/'\
        u'yearly/chart/?page={}&view=releasedate&view2=domestic&yr={}&p=.htm'.format(page,year)
    return requests.get(Mojo_url)
def Load_Data_mojo_yearly_intl(year):
    Mojo_url = u'http://www.boxofficemojo.com/'\
        u'yearly/chart/?view2=worldwide&yr={}&p=.htm'.format(year)
    return requests.get(Mojo_url)
def Load_Data_mojo_movie_intl(M_id):
    Mojo_url = u'http://www.boxofficemojo.com{}&page=intl'.format(M_id)
    return requests.get(Mojo_url)
def Load_Data_mojo_movie(M_id):
    Mojo_url = u'http://www.boxofficemojo.com{}'.format(M_id)
    return requests.get(Mojo_url)

#Daum crawler
def load_apikeys():
    fp = open("private.json","r")
    res = json.load(fp)
    return res
def load_data_daum(query):
    key = load_apikeys()['daum_key']
    try :
        url = "https://apis.daum.net/contents/movie?apikey={}&q={}&output=json".format(key, query)
    except UnicodeEncodeError :
        query2 = re.match('[a-zA-Z\s]+',query)
        url = "https://apis.daum.net/contents/movie?apikey={}&q={}&output=json".format(key, query2.group())
    except Exception as err:
        print(err)
#    print(url)
    return requests.get(url)
def select_daum(query, movie):
    res = load_data_daum(query)
#    print(res)
    data = json.loads(res.content)
    #검색결과가 0개일때, 검색하는 제목에서 영어랑 띄어쓰기만 필터링해서 (ex. Godzilla (2014) -> Godzilla) 재검색
    if data['channel']['totalCount'] == 0:
        try : 
            query2 = re.match('[a-zA-Z\s]+',query)
            if query2.group() == query:
                return -2
            else:
                return select_daum(query2.group(), movie)
        except AttributeError:
            return -2
        except Exception as err:
            print(err)
            return -2
    #검색결과가 1개일때 결과값 출력
    elif data['channel']['totalCount'] == 1:
        return data['channel']["item"][0]
    #검색결과가 여러개일때, 개봉일을 비교하여 같은경우 출력, 개봉일이 같은게 없는경우 출력안함.
    else :
        mlist = []
        try:
            m = movie['realease_kor'].split('/')
            m = '{0:02d}{1:02d}{2:02d}'.format(int(m[2]),int(m[0]),int(m[1]))
            for item in data['channel']['item']:
                try:
                    d = item['open_info'][0]['content'].split('.')
                    d = '{0:02d}{1:02d}{2:02d}'.format(int(d[0][-2:]),int(d[1]),int(d[2]))
                    if d == m :
                        mlist.append(item)
                except:
                    pass
            if len(mlist) == 1:
                return mlist[0]
            else :
                if len(mlist) != 0 :print(len(mlist))
                return -1
        except Exception as err:
            print(err)
            return -1

In [10]:
print(movie_data[0].keys())

['m_name_k', 'grade', u'Oversea', u'gross_kor', 'daum_page', u'year', 'genre', 'viewing_time', u'realease_kor', 'director_k', u'Webpage', u'Mname', u'Domestic', u'budget', u'release_usa', 'actor_k', u'runtime', u'rating_usa']


In [11]:
movie_data = db.collection.find()
arr_movie = []
err_movie = []
for arr_num, movie in enumerate(movie_data):
    res = select_daum(movie['Mname'],movie)
    dict_movie = {
        'm_name_k' : 'NA',
        'grade' : 'NA',
        'viewing_time' : -1,
        'genre' : 'NA',
        'director_k' : 'NA',
        'actor_k' : 'NA'
    }
    if res == -1 :
        err = "multiple : {}".format(movie['Mname'])
        err_movie.append(err)
    elif res == -2 : 
        err = "none : {}".format(res)
        err_movie.append(err)
    else :
        if res['open_info'][2]['content'] == '' :  res['open_info'][2]['content'] = '-1 '
        dict_movie = {
            'm_name_k' : res['title'][0]['content'],
            'grade' : res['open_info'][1]['content'],
            'viewing_time' : int(res['open_info'][2]['content'][0:-1]),
            'genre' : [X["content"] for X in res["genre"]],
            'director_k' : [(X["content"], X["link"]) for X in res["director"]],
            'actor_k' : [(X["content"], X["link"]) for X in res["actor"]],
            'daum_page' : res['title'][0]['link']
        }
    movie_data[arr_num].update(dict_movie)
fp = open("movie_added_daum.json", "w")
json.dump(movie_data, fp, indent = 4)
fp.close()

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


In [12]:
#몽고DB
db = Database()
for movie in movie_data:
    db.collection.update_one({'Mname' : movie['Mname']}, {"$set":movie}, upsert=True)

In [480]:
import time
#영화 intl 페이지 -> 한국매출 / 개봉일 정보.
def load_data_mojo_movie_intl(M_id):
    Mojo_url = u'http://www.boxofficemojo.com{}&page=intl'.format(M_id)
    req = requests.get(Mojo_url)

    bsObj = BeautifulSoup(req.content,'html.parser')
    line_kor = bsObj.find('a', text = 'South Korea')
    if line_kor :
        for iter_kor in line_kor.parents:
            if re.match('<tr ',str(iter_kor)) :
                break
        gross_kor = iter_kor.find_all(text = re.compile('\$'))
        try:
            gross_kor = int(re.sub("[^0-9]","",gross_kor[-1]))
        except IndexError:
            gross_kor = 'NA'
        release_kor = iter_kor.find_all(text = re.compile('\d+\/\d+/\d'))[0]
    else :
        gross_kor = 'NA'
        release_kor = 'NA'
    return {
        'gross_kor' : gross_kor,
        'release_kor' : release_kor
    }
#영화 Main
def load_data_mojo_movie_main(M_id):
    Mojo_url = u'http://www.boxofficemojo.com{}&page=main'.format(M_id)
    req = requests.get(Mojo_url)
    bsObj = BeautifulSoup(req.content,'html.parser')
    #content = bsObj.find('td', attrs = {'valign' : 'top', 'align':'center'}).next_sibling
    tables = bsObj.find_all('table')
    for i in tables:
        if re.search(re.compile("Domestic Total Gross"),i.get_text())!=None:
            content = i
            break
    re_dir = re.compile('.+Director.+id=.+') #감독 추출용 표현식
    re_act = re.compile('.+Actor.+id=.+') #배우 추출용
    re_name = re.compile('[0-9a-zA-Z\., ]+[0-9a-zA-Z\.]') #이름 추출용(이름 외엔 지우는..)

    director = {}
    actor = {}
    franchise = {}

    #미국매출, 제작비
    first_td = bsObj.find_all('td')
    for cell in first_td:
        if re.search('Domestic', str(cell)):
            break
    try :
        gross_us = str(re.findall(".*Domestic Total.*\$([\d,]+).*Distributor:.*", str(cell))[0])
        gross_us = int(re.sub("[^0-9]","",gross_us))
    except IndexError :
        gross_us = 'NA'
    project_budget = re.findall(".*Production Budget.*\$([\d ,a-zA-Z\.]+).*</b>.*", str(cell))
    if project_budget :
        project_budget = project_budget[0].split(" ")
        if len(project_budget) == 2 :
            unit = project_budget[1]
            if unit == 'million' :
                unit = 1000000.
            else :
                print("Need to add new unit : {}".format(unit))
            project_budget = int(float(re.sub("[^0-9\.]","",project_budget[0])) * unit)
        else :
            project_budget = int(re.sub("[^0-9\.]","",project_budget[0]))
    else :
        project_budget = 'NA'

    #The Players부분 뽑아내기.
    people_links = bsObj.find_all('a', attrs = {"href" : re.compile('\/people\/[^"]')})

    #감독 추출
    for line in people_links:
        if re.search(re_dir, str(line)):
            break
    tag_directors = line.parent.find_all('a')
    #링크 있는 감독
    for iter_dir in tag_directors:
        name = re.match(re_name, iter_dir.get_text()).group()
        name = re.sub('\.', '', name)
        name = re.sub(',','', name)
        director[name] = iter_dir.attrs['href']
    #링크 없는 감독
    names = director.keys()
    for name in line.parent.parent.strings:
        name = re.sub('\.', '', name)
        name = re.sub(',','', name)
        if not(name in names):
            actor[name] = 'NA'
            
    #배우 추출.
    for line in people_links:
        if re.search(re_act, str(line)):
            break
    tag_actors = line.parent.find_all('a')
    #링크 있는 배우들
    for iter_act in tag_actors:
        name = re.match(re_name, iter_act.get_text()).group()
        name = re.sub('\.', '', name)
        name = re.sub(',','', name)
        actor[name] = iter_act.attrs['href']
    #링크 없는 배우들
    names = actor.keys()
    for name in line.parent.parent.strings:
        name = re.sub('\.', '', name)
        name = re.sub(',','', name)
        if not(name in names):
            actor[name] = 'NA'

    #프랜차이즈
    divs = bsObj.find_all('div', attrs = {'class' : 'mp_box_tab'})
    for iter_div in divs :
        if re.search('.+Franchises.+',str(iter_div)):
            franchises_table = iter_div.parent
            for iter_ser in franchises_table.find_all('a', text = re.compile('Series:')):
                name = iter_ser.get_text()
                name = re.sub('\.', '', name)
                name = re.sub(',','', name)
                franchise[name] = iter_ser.attrs['href']
            break
        
    return {
        'gross_us' : gross_us,
        'project_budget' : project_budget,
        'director' : director,
        'actor' : actor,
        'franchise' : franchise
    }

def load_data_mojo_filmography(name, page):
    mojo_url = u'http://www.boxofficemojo.com{}'.format(page)
    print(mojo_url)
    res = requests.get(mojo_url)
    bsObj = BeautifulSoup(res.content)
    table = bsObj.find('h1')
    filmography = {}
    arr_movie = table.parent.find_all('a', attrs = {'href' : re.compile('\/movies\/')})
    for iter_movie in arr_movie:
        filmography[iter_movie.get_text()] = [iter_movie.attrs['href']]
    return filmography

def load_data_mojo_franchise(name, page):
    name = re.sub('Series: ', '', name)
    mojo_url = u'http://www.boxofficemojo.com{}'.format(page)
    res = requests.get(mojo_url)
    bsObj = BeautifulSoup(res.content)
    table = bsObj.find('h1')
    table = table.parent.find('table')
    franchise = {}
    arr_movie = table.find_all('a', attrs = {'href' : re.compile('\/movies\/')})
    arr_movie_rel = re.findall(re.compile('\d{1,2}\/\d{1,2}\/\d{1,2}'),str(table.parent.contents))
    for iter_movie in arr_movie:
        franchise[iter_movie.get_text()] = [iter_movie.attrs['href']]
    return franchise
#배우 감독 필모그래피 매출, 개봉일 작성
def load_filmography(dict_web):
    global N
    print (N)
    dict_film = {}
    for name in dict_web.keys()[N:]:
        if dict_web[name] != 'NA' :
            dict_film[name] = load_data_mojo_filmography(name, dict_web[name])
            for moviename in dict_film[name]:
                dict_movie = db.collection.find_one({'Mname': moviename })
                if dict_movie != None:
                    dict_film[name][moviename].append(dict_movie['release_kor'])
                    dict_film[name][moviename].append(dict_movie['gross_kor'])
                else :
                    res = load_data_mojo_movie_intl(dict_film[name][moviename][0])
                    dict_film[name][moviename].append(res['release_kor'])
                    dict_film[name][moviename].append(res['gross_kor'])
                    db.collection.insert_one({'Mname': moviename, 'release_kor': res['release_kor'], 'gross_kor': res['gross_kor']})
        else :
            dict_film[name] = ['NA']
        N += 1
    return dict_film

In [412]:
#real code
#초기화
N = 0
db = Database()
movies = db.collection.find()
dict_web_actor = {}
dict_web_director = {}
dict_web_franchise = {}
dict_film_actor = {}
dict_film_director = {}
dict_film_franchise = {}

In [438]:
#real code
#각 영화에 대한 scraping 실행.
db = Database()
movies = db.collection.find()
print(len(dict_web_actor), len(dict_web_director), len(dict_web_franchise))
for iter_movie in movies[N:]:
    try:
        sys.stdout.write('{},{} '.format(N, iter_movie[u'Mname']))
    except:
        pass
    main = load_data_mojo_movie_main(iter_movie[u'Webpage'])
    for k in main['actor'].keys():
        dict_web_actor[k] = main['actor'][k]
    for k in main['director'].keys():
        dict_web_director[k] = main['director'][k]
    for k in main['franchise'].keys():
        dict_web_franchise[k] = main['franchise'][k]
    main.update(load_data_mojo_movie_intl(iter_movie[u'Webpage']))
    db.collection.update_one({'_id' : iter_movie['_id']}, {'$set' : main})
    N+=1
    time.sleep(0.5)

847
1642 392 96
Live By Night God's Not Dead 2 El Clan Nine Lives (2016) Eye in the Sky Regression Norm of the North Pride and Prejudice and Zombies The Infiltrator The Take (2016) Hardcore Henry Love & Friendship Sing Street Jackie The Beatles: Eight Days a Week - The Touring Years The Man Who Knew Infinity Ratchet & Clank The Lobster (US Release) A Bigger Splash Captain Fantastic Hands of Stone Men & Chicken Where to Invade Next A Man Called Ove Everybody Wants Some!! Maggie's Plan The Program (2016) Desierto Miles Ahead Equals The Dressmaker The Handmaiden Demolition The Neon Demon The Music of Strangers Operation Chromite The Wailing (Goksung) Master A Tale of Love and Darkness Tunnel Knight of Cups Cold War 2 The Age of Shadows Marguerite The Boy and the Beast Our Little Sister Things To Come High-Rise Francofonia Kingsglaive: Final Fantasy: XV The Family Fang Dheepan Oasis: Supersonic League of Gods Viva The Wave Asura: The City of Madness Louder than Bombs Born to be Blue Tumbl

In [481]:
N = 39

In [ ]:
#real code
#배우, 프랜차이즈, 감독 딕셔너리 완성
dict_film_actor = load_filmography(dict_web_actor)
N = 0
dict_film_director = load_filmography(dict_web_director)

dict_film = {}
dict_web = dict_web_franchise
for name in dict_web.keys():
    if dict_web[name] != 'NA' :
        dict_film[name] = load_data_mojo_franchise(name, dict_web[name])
        for moviename in dict_film[name]:
            dict_movie = db.collection.find_one({'Mname': moviename })
            if dict_movie != None:
                dict_film[name][moviename].append(dict_movie['realease_kor'])
                dict_film[name][moviename].append(dict_movie['gross_kor'])
            else :
                res = load_data_mojo_movie_intl(dict_film[name][moviename][0])
                dict_film[name][moviename].append(res['release_kor'])
                dict_film[name][moviename].append(res['gross_kor'])
                db.collection.insert_one({'Mname': moviename, 'release_kor': res['release_kor'], 'gross_kor': res['gross_kor']})
    else :
        dict_film[name] = ['NA']
dict_film_franchise = dict_film

connection = pymongo.MongoClient("localhost", 27017)
db = connection["DSS4_SmallP"]
actordb = db["actor"]
dirdb = db["director"]
frandb = db["franchise"]

39
http://www.boxofficemojo.com/people/chart/?view=Actor&id=michaelbjordan.htm
http://www.boxofficemojo.com/people/chart/?view=Actor&id=paulapatton.htm
http://www.boxofficemojo.com/people/chart/?view=Actor&id=karengillan.htm
http://www.boxofficemojo.com/people/chart/?view=Actor&id=lukashaas.htm
http://www.boxofficemojo.com/people/chart/?view=Actor&id=davefranco.htm
http://www.boxofficemojo.com/people/chart/?view=Actor&id=lucasblack.htm
http://www.boxofficemojo.com/people/chart/?view=Actor&id=ashleybell.htm
http://www.boxofficemojo.com/people/chart/?view=Actor&id=charlieday.htm
http://www.boxofficemojo.com/people/chart/?view=Actor&id=stellanskarsgard.htm
http://www.boxofficemojo.com/people/chart/?view=Actor&id=johnnydepp.htm
http://www.boxofficemojo.com/people/chart/?view=Actor&id=ryanguzman.htm
http://www.boxofficemojo.com/people/chart/?view=Actor&id=thandienewton.htm
http://www.boxofficemojo.com/people/chart/?view=Actor&id=colinegglesfield.htm
http://www.boxofficemojo.com/people/chart

In [410]:
###############################################################################################
#여기부턴 테스트&디버깅용

import sys
for i in range(10):
    sys.stdout.write('a')

aaaaaaaaaa

In [367]:
#test
re_name = re.compile('[a-zA-Z\., ]+[a-zA-Z\.]') #이름 추출용(이름 외엔 지우는..)
a = '<font size="2">Xavier Samuel'
print(re.match(re_name, a))

None


In [436]:
iter_movie

{u'Domestic': u'$10.3',
 u'Mname': u'Live By Night',
 u'Oversea': u'$11.3',
 u'Webpage': u'/movies/?id=livebynight.htm',
 u'_id': ObjectId('58b49f1ff4758a5a0ee397cc'),
 u'actor_k': [[u'\ubca4 \uc560\ud50c\ub809',
   u'http://movie.daum.net/person/main?personId=617'],
  [u'\uc5d8\ub974 \ud328\ub2dd',
   u'http://movie.daum.net/person/main?personId=95465'],
  [u'\ube0c\ub80c\ub4e0 \uae00\ub9ac\uc2a8',
   u'http://movie.daum.net/person/main?personId=10671'],
  [u'\ud06c\ub9ac\uc2a4 \uba54\uc2dc\ub098',
   u'http://movie.daum.net/person/main?personId=116446'],
  [u'\uc2dc\uc5d0\ub098 \ubc00\ub7ec',
   u'http://movie.daum.net/person/main?personId=94391'],
  [u'\uc870 \uc0d0\ub2e4\ub098',
   u'http://movie.daum.net/person/main?personId=17260'],
  [u'\ud06c\ub9ac\uc2a4 \ucfe0\ud37c',
   u'http://movie.daum.net/person/main?personId=19276']],
 u'budget': u'N/A',
 u'daum_page': u'http://movie.daum.net/moviedb/main?movieId=106273',
 u'director_k': [[u'\ubca4 \uc560\ud50c\ub809',
   u'http://movie

In [468]:
name = 'Michael B Jordan'
name = re.match(re.compile('[A-Za-z]+'), name).group()
mojo_url = 'http://www.boxofficemojo.com/people/chart/?view=Actor&id=michaelbjordan.htm'
print(mojo_url)
res = requests.get(mojo_url)
bsObj = BeautifulSoup(res.content)
table = bsObj.find('h1')
filmography = {}
arr_movie = table.parent.find_all('a', attrs = {'href' : re.compile('\/movies\/')})
for iter_movie in arr_movie:
    filmography[iter_movie.get_text()] = [iter_movie.attrs['href']]

http://www.boxofficemojo.com/people/chart/?view=Actor&id=michaelbjordan.htm


In [469]:
print(table)

<h1>Michael B. Jordan</h1>
